Датасет с фильмами: https://grouplens.org/datasets/movielens/

In [1]:
import numpy as np
import pandas as pd
import time

ratings_df = pd.read_csv('./ratings.csv')
print('Количество пользователей: {}'.format(len(ratings_df['userId'].unique())))
print('Количество фильмов: {}'.format(len(ratings_df['movieId'].unique())))
print('Размерность датасета: {}'.format(ratings_df.shape))

ratings_df.head()

Количество пользователей: 610
Количество фильмов: 9724
Размерность датасета: (100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [2]:
# Отберём для тестирования часть набора и отмасштабируем для удобства идентификаторы фильмов так,
# чтобы они начинались с 1 и заканчивались на n_movies:

n = 100000
ratings_df_sample = ratings_df[:n]

n_users = len(ratings_df_sample['userId'].unique())
n_movies = len(ratings_df_sample['movieId'].unique())
(n_users, n_movies)


(610, 9569)

In [3]:
from surprise import Reader, Dataset, SVD, NMF
from surprise.model_selection import train_test_split
from surprise import accuracy

# создание объекта класса Reader
reader = Reader(rating_scale=(1, 10))

# создание объекта класса Dataset
dataset = Dataset.load_from_df(ratings_df_sample[['userId', 'movieId', 'rating']], reader)

# разбиение данных на обучающую и тестовую выборки
trainset, testset = train_test_split(dataset, test_size = 0.1)

# создание экземпляра класса SVD
# model = SVD()
model = NMF()

# обучение модели на обучающей выборке
model.fit(trainset)

# предсказание рейтингов на тестовой выборке
predictions = model.test(testset)

# оценка качества модели
print('RMSE:', accuracy.rmse(predictions))
print('MAE:', accuracy.mae(predictions))

R_pred_surprise = np.zeros((n_users, n_movies))
for u in range(n_users):
    for m in range(n_movies):
        R_pred_surprise[u][m] = model.predict(u, m).est
        
pd.DataFrame(np.round(R_pred_surprise, 2))

RMSE: 0.9119
RMSE: 0.9119108254006894
MAE:  0.6977
MAE: 0.6977321395955607


,0,1,2,3,4,5,6,7,8,9,...,9559,9560,9561,9562,9563,9564,9565,9566,9567,9568
0,3.5,3.50,3.50,3.50,3.50,3.50,3.50,3.50,3.50,3.50,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
1,3.5,4.45,4.01,3.93,2.90,3.61,4.60,3.65,2.79,4.14,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
2,3.5,3.85,3.34,3.71,2.57,2.83,3.72,2.51,2.41,2.93,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
3,3.5,1.93,1.21,1.99,1.00,1.42,1.67,1.35,1.00,1.92,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
4,3.5,3.52,3.21,2.63,2.40,2.12,3.61,2.14,1.74,3.19,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,3.5,3.67,3.23,3.12,2.42,2.66,3.86,3.14,1.70,2.72,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
606,3.5,3.86,3.34,3.28,2.40,3.05,4.16,3.21,1.62,3.35,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
607,3.5,4.09,3.39,3.32,3.12,3.15,3.80,3.15,2.86,3.55,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
608,3.5,3.75,3.22,2.77,2.61,2.58,3.43,2.58,2.07,2.77,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5


In [3]:
movie_ids = ratings_df_sample['movieId'].unique()

def scale_movie_id(movie_id):
    scaled = np.where(movie_ids == movie_id)[0][0] + 1
    return scaled

ratings_df_sample['movieId'] = ratings_df_sample['movieId'].apply(scale_movie_id)
ratings_df_sample.head()

C:\Users\User\AppData\Local\Temp\ipykernel_6600\3425480383.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings_df_sample['movieId'] = ratings_df_sample['movieId'].apply(scale_movie_id)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,2,4.0,964981247
2,1,3,4.0,964982224
3,1,4,5.0,964983815
4,1,5,5.0,964982931


In [23]:
from sklearn.model_selection import train_test_split
# обучающий и тестовый наборы
train_data, test_data = train_test_split(ratings_df_sample, test_size=0.2)

In [5]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from math import sqrt

# метрика RMSE (Root Mean Square Error, среднеквадратическая ошибка)

def rmse(prediction, ground_truth):
    # оставим оценки алгоритма только для соответствующего набора данных
    prediction = np.nan_to_num(prediction)[ground_truth.nonzero()].flatten()
    # оставим оценки пользователя только для соответствующего набора данных
    ground_truth = np.nan_to_num(ground_truth)[ground_truth.nonzero()].flatten()
    
    mse = mean_squared_error(prediction, ground_truth)
    return sqrt(mse)

In [24]:
# Сформируем матрицы размера (n_users, n_movies) для обучающего и тестового наборов, в которых
# элемент в ячейке [i, j] отражает оценку i-го пользователя j-му фильму
    
train_data_matrix = np.zeros((n_users, n_movies))
for line in train_data.itertuples():
    train_data_matrix[line[1] - 1, line[2] - 1] = line[3]
    
test_data_matrix = np.zeros((n_users, n_movies))
for line in test_data.itertuples():
    test_data_matrix[line[1] - 1, line[2] - 1] = line[3]

In [26]:
train_data_matrix

array([[4. , 4. , 4. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 2. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 5. , ..., 4. , 3. , 3.5]])

In [7]:
from  sklearn.metrics.pairwise import pairwise_distances
# ключевое в алгоритме -- мера близости
# считаем косинусное расстояние для пользователей и фильмов (построчно и поколоночно соотвественно)

# Valid values for metric are:
# [‘cityblock’, ‘cosine’, ‘euclidean’, ‘l1’, ‘l2’, ‘manhattan’]
# [‘nan_euclidean’]
# [‘braycurtis’, ‘canberra’, ‘chebyshev’, ‘correlation’, ‘dice’,
#  ‘hamming’, ‘jaccard’, ‘kulsinski’, ‘mahalanobis’, ‘minkowski’,
#  ‘rogerstanimoto’, ‘russellrao’, ‘seuclidean’, ‘sokalmichener’,
#  ‘sokalsneath’, ‘sqeuclidean’, ‘yule’]

# подходят евклидово, косинусное, корреляция 

user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [8]:
# User-based collaborative filtering
def predict_user(top):
    # Структура для хранения для каждого пользователя оценки фильмов top наиболее похожих на него пользователей:
    # top_similar_ratings[0][1] - оценки всех фильмов одного из наиболее похожих пользователей на пользователя с ид 0.
    # Здесь 1 - это не ид пользователя, а просто порядковый номер.
    top_similar_ratings = np.zeros((n_users, top, n_movies))

    for i in range(n_users):
        # Для каждого пользователя необходимо получить наиболее похожих пользователей:
        # Нулевой элемент не подходит, т.к. на этом месте находится похожесть пользователя самого на себя
        top_sim_users = user_similarity[i].argsort()[1:top + 1]
        # берём только оценки из "обучающей" выборки 
        top_similar_ratings[i] = train_data_matrix[top_sim_users]

    pred = np.zeros((n_users, n_movies))
    for i in range(n_users):
        pred[i] = top_similar_ratings[i].sum(axis=0) / top
    
    return pred

pred_user = predict_user(9)
print('Основанная на пользователях фильтрация, RMSE: ', rmse(pred_user, test_data_matrix))

Основанная на пользователях фильтрация, RMSE:  2.7619432850579213


In [9]:
def predict_item(top):
    top_similar_ratings = np.zeros((n_movies, top, n_users))

    for i in range(n_movies):
        top_sim_movies = item_similarity[i].argsort()[1:top + 1]
        top_similar_ratings[i] = train_data_matrix.T[top_sim_movies]
        
    pred = np.zeros((n_movies, n_users))
    for i in range(n_movies):
        pred[i] = top_similar_ratings[i].sum(axis=0) / top
    
    return pred.T

pred_item = predict_item(9)
print('Основанная на пользователях фильтрация, RMSE: ', rmse(pred_item, test_data_matrix))

Основанная на пользователях фильтрация, RMSE:  2.9322275114684304


In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
axis_x = ['cosine', 'euclidean', 'correlation']
graph1 = []
graph2 = []
graph01 = []
graph02 = []

for metr in axis_x:
    start_time = time.time()
    global user_similarity
    user_similarity = pairwise_distances(train_data_matrix, metric=metr)
    global item_similarity
    item_similarity = pairwise_distances(train_data_matrix.T, metric=metr)
    end_time = time.time()
    e_time = (end_time - start_time)
    print("Время на инициализацию с метрикой {}: {} c".format(metr, e_time))
    
    e_time1 = 0
    e_time2 = 0
    
    for i in range(10):
        start_time = time.time()
#         for i in range(0, 100):
        predict_user(9)
        end_time = time.time()
        e_time1 += (end_time - start_time)
        
        start_time = time.time()
#         for i in range(0, 100):
        predict_item(9)
        end_time = time.time()
        e_time2 += (end_time - start_time)
        
    graph1.append(e_time1 / 10)
    graph2.append(e_time2 / 10)
    
    pred_i = predict_item(9)
    pred_u = predict_user(9)
    
    graph01.append(rmse(pred_u, test_data_matrix))
    graph02.append(rmse(pred_i, test_data_matrix))

plt.plot(axis_x, graph01, marker='s', color='r', label="по пользователям");
plt.plot(axis_x, graph02, marker='s', color='g', label="по предметам");
plt.legend()
plt.xlabel('Метрика')
plt.ylabel('Ошибка RMSE');

In [ ]:
plt.plot(axis_x, graph1, marker='s', color='r', label="по пользователям, c");
plt.plot(axis_x, graph2, marker='s', color='g', label="по предметам, c");
plt.legend()
plt.xlabel('Метрика')
plt.ylabel('Время');

In [ ]:
axis_x = ['cosine', 'euclidean', 'correlation']
graph1 = []
graph2 = []

ns = [1000, 5000, 10000, 20000, 50000, 100000]

for n in ns:
    global train_data_matrix
    global test_data_matrix
    global item_similarity
    global user_similarity
    
    ratings_df_sample = ratings_df[:n]
    n_users = len(ratings_df_sample['userId'].unique())
    n_movies = len(ratings_df_sample['movieId'].unique())
    print(n_users, n_movies)
    movie_ids = ratings_df_sample['movieId'].unique()
    ratings_df_sample['movieId'] = ratings_df_sample['movieId'].apply(scale_movie_id)
    train_data, test_data = train_test_split(ratings_df_sample, test_size=0.2)
    train_data_matrix = np.zeros((n_users, n_movies))
    for line in train_data.itertuples():
        train_data_matrix[line[1] - 1, line[2] - 1] = line[3]
    test_data_matrix = np.zeros((n_users, n_movies))
    for line in test_data.itertuples():
        test_data_matrix[line[1] - 1, line[2] - 1] = line[3]
    
    start_time = time.time()
    user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
    item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')
    end_time = time.time()
    e_time = (end_time - start_time)
    print("Время на инициализацию с n = {}: {} c".format(n, e_time))
    
    e_time1 = 0
    e_time2 = 0
    
    for i in range(10):
        start_time = time.time()
#         for i in range(0, 100):
        predict_user(5)
        end_time = time.time()
        e_time1 += (end_time - start_time)
        
        start_time = time.time()
#         for i in range(0, 100):
        predict_item(5)
        end_time = time.time()
        e_time2 += (end_time - start_time)
        
    graph1.append(e_time1 / 10)
    graph2.append(e_time2 / 10)

plt.plot(ns, graph1, marker='s', color='r', label="по пользователям, c");
plt.plot(ns, graph2, marker='s', color='g', label="по предметам, c");
plt.legend()
plt.xlabel('Количество строк в наборе данных')
plt.ylabel('Время');

In [1]:
import numpy as np
import pandas as pd

In [2]:
k = 10 # максимальная оценка

movies = ['Фантазия', 'ВАЛЛ-И', 'Пиноккио', 'Бемби' , 'Шрэк', 'Дамбо', 'Спасатели', 'Геркулес', 'Кунг-фу Панда']
m_movies = len(movies)

users = ['Андрей', 'Аня', 'Алиса', 'Ваня', 'Леша', 'Оксана', 'Саша', 'Паша', 'Сеня', 'Гриша']        
n_users = len(users)

In [4]:
np.random.seed(42)

N = np.random.randint(50, 60) # сколько оценок будет поставлено

ind_users, ind_movies, rating = [], [], []
user_movie = [] # чтобы пара user-movie не повторялись

for _ in range(N):
    user = np.random.randint(0, n_users)
    movie = np.random.randint(0, m_movies)
    if not [user, movie] in user_movie:
        ind_users.append(user)
        ind_movies.append(movie)
        rating.append(np.random.randint(1, k)) # случайная оценка пользователя фильму
        user_movie.append([user, movie])        
N = len(user_movie)

data = {'userId': ind_users, 'movieId': ind_movies, 'rating': rating}
R_train = pd.DataFrame(data = data)
R_train.head(3)


,userId,movieId,rating
0,3,7,5
1,6,2,7
2,7,4,4


In [50]:
User_item_matrix = train_data.pivot(columns = 'movieId', index = 'userId', values = 'rating')

In [51]:
User_item_matrix = R_train.pivot(columns = 'movieId', index = 'userId', values = 'rating')
User_item_matrix.rename(columns = dict(zip(User_item_matrix.columns, movies)), inplace = True)
User_item_matrix.set_index(pd.Index(users), inplace=True)
User_item_matrix

movieId,Фантазия,ВАЛЛ-И,Пиноккио,Бемби,Шрэк,Дамбо,Спасатели,Геркулес,Кунг-фу Панда
Андрей,NaN,NaN,NaN,4.0,7.0,NaN,NaN,NaN,7.0
Аня,7.0,4.0,NaN,NaN,8.0,6.0,NaN,4.0,NaN
Алиса,4.0,NaN,NaN,NaN,3.0,NaN,NaN,6.0,NaN
Ваня,1.0,NaN,NaN,NaN,NaN,NaN,8.0,5.0,NaN
Леша,6.0,NaN,NaN,NaN,NaN,4.0,7.0,NaN,NaN
Оксана,NaN,9.0,NaN,NaN,2.0,NaN,NaN,9.0,NaN
Саша,1.0,4.0,7.0,9.0,9.0,8.0,NaN,5.0,8.0
Паша,NaN,NaN,3.0,NaN,4.0,2.0,9.0,3.0,NaN
Сеня,3.0,9.0,7.0,NaN,1.0,NaN,NaN,1.0,4.0
Гриша,NaN,NaN,NaN,6.0,2.0,NaN,9.0,NaN,9.0


In [40]:
n_users

610

In [5]:
User_item_matrix = R_train.pivot(columns = 'movieId', index = 'userId', values = 'rating')
User_item_matrix.rename(columns = dict(zip(User_item_matrix.columns, movies)), inplace = True)
User_item_matrix.set_index(pd.Index(users), inplace=True)
User_item_matrix

movieId,Фантазия,ВАЛЛ-И,Пиноккио,Бемби,Шрэк,Дамбо,Спасатели,Геркулес,Кунг-фу Панда
Андрей,NaN,NaN,NaN,4.0,7.0,NaN,NaN,NaN,7.0
Аня,7.0,4.0,NaN,NaN,8.0,6.0,NaN,4.0,NaN
Алиса,4.0,NaN,NaN,NaN,3.0,NaN,NaN,6.0,NaN
Ваня,1.0,NaN,NaN,NaN,NaN,NaN,8.0,5.0,NaN
Леша,6.0,NaN,NaN,NaN,NaN,4.0,7.0,NaN,NaN
Оксана,NaN,9.0,NaN,NaN,2.0,NaN,NaN,9.0,NaN
Саша,1.0,4.0,7.0,9.0,9.0,8.0,NaN,5.0,8.0
Паша,NaN,NaN,3.0,NaN,4.0,2.0,9.0,3.0,NaN
Сеня,3.0,9.0,7.0,NaN,1.0,NaN,NaN,1.0,4.0
Гриша,NaN,NaN,NaN,6.0,2.0,NaN,9.0,NaN,9.0


In [6]:
mu = R_train['rating'].mean()
mu

5.441860465116279

In [7]:
bu = np.zeros(n_users)
bm = np.zeros(m_movies)
print(bu.shape, bm.shape)

(10,) (9,)


In [8]:
d = 5  # главных компонент
pu = np.random.normal(0, 0.1, (n_users, d))
qm = np.random.normal(0, 0.1, (m_movies, d))
print(pu.shape, qm.shape)

(10, 5) (9, 5)


In [9]:
epoch = 5
gamma = 0.02
lmbda = 0.03

for _ in range(epoch):
    
    for index in range(R_train.shape[0]):

        user_id = R_train['userId'][index] 
        movie_id = R_train['movieId'][index] 
        rating = R_train['rating'][index]

        err = rating - (mu + bu[user_id] + bm[movie_id] + qm[movie_id] @ pu[user_id])

        bu[user_id] += gamma * (err - lmbda * bu[user_id]) 
        bm[movie_id] += gamma * (err - lmbda * bm[movie_id]) 
        pu[user_id] += gamma * (err * qm[movie_id] - lmbda * pu[user_id])
        qm[movie_id] += gamma * (err * pu[user_id] - lmbda * qm[movie_id])

In [10]:
R_pred = np.zeros((n_users, m_movies))

for user in range(n_users):
    for movie in range(m_movies):
        R_pred[user][movie] = round(mu + bu[user] + bm[movie] + qm[movie] @ pu[user], 2)

pd.DataFrame(R_pred, users, movies)

,Фантазия,ВАЛЛ-И,Пиноккио,Бемби,Шрэк,Дамбо,Спасатели,Геркулес,Кунг-фу Панда
Андрей,4.78,5.93,5.61,5.70,5.10,5.41,6.54,5.26,6.03
Аня,4.88,5.97,5.70,5.81,5.20,5.50,6.66,5.33,6.15
Алиса,4.41,5.57,5.43,5.53,4.60,5.16,6.17,4.91,5.83
Ваня,4.40,5.54,5.36,5.45,4.74,5.11,6.20,4.90,5.72
Леша,4.71,5.80,5.51,5.59,5.05,5.29,6.49,5.17,5.93
Оксана,4.99,6.23,5.92,6.00,5.20,5.70,6.71,5.52,6.34
Саша,5.08,6.27,6.10,6.23,5.54,5.85,6.92,5.59,6.45
Паша,4.18,5.28,4.97,5.08,4.46,4.79,5.94,4.63,5.45
Сеня,4.03,5.24,5.05,5.09,4.26,4.76,5.78,4.57,5.38
Гриша,4.94,6.12,5.84,5.96,5.16,5.65,6.67,5.43,6.30


In [16]:
dir(trainset)


In [18]:
from surprise import Reader, Dataset, SVD, NMF
from surprise.model_selection import train_test_split
from surprise import accuracy

# создание объекта класса Reader
reader = Reader(rating_scale=(1, 10))

# создание объекта класса Dataset
dataset = Dataset.load_from_df(R_train[['userId', 'movieId', 'rating']], reader)

# разбиение данных на обучающую и тестовую выборки
trainset, testset = train_test_split(dataset, test_size = 0.1)

# создание экземпляра класса SVD
# model = SVD()
model = SVD()

# обучение модели на обучающей выборке
model.fit(trainset)

# предсказание рейтингов на тестовой выборке
predictions = model.test(testset)

# оценка качества модели
print('RMSE:', accuracy.rmse(predictions))
print('MAE:', accuracy.mae(predictions))

R_pred_surprise = np.zeros((n_users, m_movies))
for u in range(n_users):
    for m in range(m_movies):
        R_pred_surprise[u][m] = model.predict(u, m).est
        
pd.DataFrame(np.round(R_pred_surprise, 2), users, movies)

RMSE: 2.6394
RMSE: 2.639401245500767
MAE:  2.1515
MAE: 2.1515325247617225


,Фантазия,ВАЛЛ-И,Пиноккио,Бемби,Шрэк,Дамбо,Спасатели,Геркулес,Кунг-фу Панда
Андрей,5.25,5.82,5.48,5.50,5.53,5.46,6.47,4.88,5.83
Аня,5.74,5.33,5.48,5.88,6.09,5.70,6.46,5.01,6.04
Алиса,5.16,5.95,5.58,5.72,5.30,5.34,6.49,5.43,6.04
Ваня,4.23,5.75,5.41,5.50,4.67,5.22,6.64,4.90,5.63
Леша,5.49,5.88,5.49,5.75,5.13,5.05,6.49,5.35,5.75
Оксана,5.14,6.97,6.11,5.85,4.35,5.51,6.87,6.18,5.96
Саша,6.07,5.80,5.95,6.90,6.77,6.52,7.03,5.65,6.92
Паша,4.74,5.45,4.67,5.26,4.65,4.42,6.33,4.46,5.56
Сеня,3.93,6.61,5.57,4.98,3.25,4.62,5.91,3.83,5.03
Гриша,5.21,6.49,6.05,5.91,4.39,5.43,7.31,5.39,6.70


In [13]:
from surprise import Reader, Dataset, SVD, NMF, SVDpp
from surprise.model_selection import train_test_split
from surprise import accuracy

# создание объекта класса Reader
reader = Reader(rating_scale=(1, 10))

# создание объекта класса Dataset
dataset = Dataset.load_from_df(ratings_df_sample[['userId', 'movieId', 'rating']], reader)

# разбиение данных на обучающую и тестовую выборки
trainset, testset = train_test_split(dataset, test_size = 0.1)

# создание экземпляра класса SVD
# model = SVD()
model = SVD()

# обучение модели на обучающей выборке
model.fit(trainset)

# предсказание рейтингов на тестовой выборке
predictions = model.test(testset)

# оценка качества модели
print('RMSE:', accuracy.rmse(predictions))
print('MAE:', accuracy.mae(predictions))

R_pred_surprise = np.zeros((n_users, n_movies))
for u in range(n_users):
    for m in range(n_movies):
        R_pred_surprise[u][m] = model.predict(u, m).est
        
pd.DataFrame(np.round(R_pred_surprise, 2))

RMSE: 0.8786
RMSE: 0.878581348874822
MAE:  0.6758
MAE: 0.6757748125756766


,0,1,2,3,4,5,6,7,8,9,...,9559,9560,9561,9562,9563,9564,9565,9566,9567,9568
0,3.50,3.90,3.28,3.95,4.00,4.25,3.61,3.99,4.07,3.65,...,3.53,3.57,3.42,3.47,3.33,3.45,3.50,3.30,3.53,3.45
1,4.19,4.53,3.89,4.44,4.81,4.83,4.11,4.86,4.50,4.34,...,4.15,4.22,4.30,4.05,4.19,4.39,4.19,4.04,4.33,4.24
2,3.60,4.09,3.66,3.98,4.10,4.24,3.54,4.19,4.20,3.61,...,3.56,3.50,3.60,3.60,3.51,3.38,3.60,3.45,3.66,3.44
3,2.50,2.60,1.89,2.99,2.65,3.44,2.83,2.84,2.85,2.66,...,2.51,2.19,2.47,2.62,2.44,2.42,2.50,2.15,2.72,2.51
4,3.29,3.67,3.31,3.69,3.07,4.02,3.16,3.61,3.52,3.07,...,3.13,3.45,3.20,3.14,3.14,3.54,3.29,3.11,2.88,2.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,3.21,3.63,3.08,3.62,3.62,3.83,3.34,3.46,3.72,3.38,...,3.04,3.24,2.83,3.21,2.95,3.09,3.21,2.97,3.27,3.13
606,3.53,3.75,3.19,3.82,3.75,4.22,3.68,4.16,3.80,3.70,...,3.68,3.48,3.55,3.57,3.59,3.64,3.53,3.57,3.68,3.47
607,3.65,3.93,3.51,3.99,4.18,4.19,3.59,4.25,4.18,3.57,...,3.52,3.67,3.45,3.59,3.57,3.56,3.65,3.23,3.78,3.87
608,3.14,3.03,2.26,3.68,4.21,4.51,3.40,3.73,3.89,2.92,...,3.44,3.49,3.19,3.12,3.17,2.96,3.14,3.20,3.32,2.77


In [11]:
from surprise import Reader, Dataset, SVD, NMF
from surprise.model_selection import train_test_split
from surprise import accuracy

# создание объекта класса Reader
reader = Reader(rating_scale=(1, 10))

# создание объекта класса Dataset
dataset = Dataset.load_from_df(ratings_df_sample[['userId', 'movieId', 'rating']], reader)

# разбиение данных на обучающую и тестовую выборки
trainset, testset = train_test_split(dataset, test_size = 0.1)

# создание экземпляра класса SVD
# model = SVD()
model = NMF()

# обучение модели на обучающей выборке
model.fit(trainset)

# предсказание рейтингов на тестовой выборке
predictions = model.test(testset)

# оценка качества модели
print('RMSE:', accuracy.rmse(predictions))
print('MAE:', accuracy.mae(predictions))

R_pred_surprise = np.zeros((n_users, n_movies))
for u in range(n_users):
    for m in range(n_movies):
        R_pred_surprise[u][m] = model.predict(u, m).est
        
pd.DataFrame(np.round(R_pred_surprise, 2))

RMSE: 0.9130
RMSE: 0.9130384649408517
MAE:  0.6960
MAE: 0.6959904942330161


,0,1,2,3,4,5,6,7,8,9,...,9559,9560,9561,9562,9563,9564,9565,9566,9567,9568
0,3.5,3.50,3.50,3.50,3.50,3.50,3.50,3.50,3.50,3.50,...,3.50,3.50,3.50,3.50,3.50,3.50,3.50,3.5,3.50,3.50
1,3.5,4.70,4.01,4.55,4.79,5.25,4.04,4.72,4.50,4.95,...,3.57,4.07,2.40,3.46,1.99,2.88,2.66,3.5,3.90,2.93
2,3.5,3.96,2.98,3.53,4.17,3.98,3.29,3.21,3.57,3.08,...,3.07,3.16,2.17,2.76,1.78,2.53,2.33,3.5,3.23,2.24
3,3.5,2.51,1.04,1.81,2.70,2.04,2.04,1.82,2.42,1.71,...,1.88,2.29,1.58,1.88,1.17,1.41,1.43,3.5,1.62,1.00
4,3.5,3.84,3.13,3.58,3.43,4.00,3.73,4.12,3.48,4.05,...,2.49,3.20,1.63,2.90,1.38,2.23,2.08,3.5,2.81,2.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,3.5,3.69,2.60,3.69,3.90,3.67,3.52,3.16,3.72,3.59,...,3.01,3.57,2.12,3.08,1.62,2.43,2.27,3.5,3.23,2.30
606,3.5,3.60,3.03,3.83,4.05,4.15,3.33,4.02,4.14,3.54,...,2.98,3.60,2.18,2.70,1.56,2.37,2.25,3.5,3.15,2.54
607,3.5,4.08,3.45,4.05,4.50,4.27,3.88,3.48,3.98,3.77,...,2.95,3.34,2.07,3.49,1.95,2.90,2.74,3.5,3.66,2.61
608,3.5,3.57,2.48,3.39,4.12,3.77,3.28,3.34,3.78,3.12,...,3.50,4.00,2.50,2.54,1.67,2.18,2.10,3.5,3.16,2.42


In [29]:
train_data_matrix

array([[4. , 4. , 4. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 2. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 5. , ..., 4. , 3. , 3.5]])

In [30]:
User_item_matrix

movieId,Фантазия,ВАЛЛ-И,Пиноккио,Бемби,Шрэк,Дамбо,Спасатели,Геркулес,Кунг-фу Панда
Андрей,NaN,NaN,NaN,4.0,7.0,NaN,NaN,NaN,7.0
Аня,7.0,4.0,NaN,NaN,8.0,6.0,NaN,4.0,NaN
Алиса,4.0,NaN,NaN,NaN,3.0,NaN,NaN,6.0,NaN
Ваня,1.0,NaN,NaN,NaN,NaN,NaN,8.0,5.0,NaN
Леша,6.0,NaN,NaN,NaN,NaN,4.0,7.0,NaN,NaN
Оксана,NaN,9.0,NaN,NaN,2.0,NaN,NaN,9.0,NaN
Саша,1.0,4.0,7.0,9.0,9.0,8.0,NaN,5.0,8.0
Паша,NaN,NaN,3.0,NaN,4.0,2.0,9.0,3.0,NaN
Сеня,3.0,9.0,7.0,NaN,1.0,NaN,NaN,1.0,4.0
Гриша,NaN,NaN,NaN,6.0,2.0,NaN,9.0,NaN,9.0
